In [1]:
import os

import subprocess

In [2]:
cmd = f"env TZ=Europe/Amsterdam date +%Y-%m-%d:%H%M"
cmd_out=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

In [3]:
cmd_out

'2019-02-03:1756\n'

In [4]:
#!git log --name-only

In [9]:
def parse_git_author(author):
    import re
    pat = r"(.*) <(.*)>"
    matchObj = re.match(pat, author)
    return matchObj.groups() if matchObj else None

#if matchObj:
# print("Matched")
# print(matchObj.groups())   
# author_userid,author_email = matchObj.groups()
# print(author_userid,author_email) 

In [11]:
author = "wgong <wen.g.gong@gmail.com>"
# author = "wgong"
print(parse_git_author(author))

('wgong', 'wen.g.gong@gmail.com')


In [60]:
def parse_git_log(log):
    lst = log.split("commit ")
    commit_log = []  # list of dict{commit:, Author:, Date:, msg:}
    for i in lst:
        if len(i) == 0: continue
        dict_log = {}
        fields = i.split('\n')
        dict_log["commit"] = fields[0]
        dict_log["Author"] = fields[1].replace("Author:", "").strip()
        dict_log["Date"] = fields[2].replace("Date:", "").strip()
        dict_log["msg"] = fields[4].strip()
        #dict_log["files"] = fields[6].strip().split()
        dict_log["files"] = [i for i in fields[5:] if len(i)]
        
        commit_log.append(dict_log)
    return commit_log

In [47]:
cmd = f"git log --name-only"
cmd_out=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

git_log = parse_git_log(cmd_out)

In [62]:
git_log[0], git_log[-1]

({'commit': 'c1260ab315b782f2bc056fa40f631ef23983a987',
  'Author': 'wgong <wen.g.gong@gmail.com>',
  'Date': 'Wed Jan 30 12:43:55 2019 -0500',
  'msg': 'add date timezone',
  'files': ['Jenkinsfile']},
 {'commit': '7a39e583c40683933767de35728f3c7d85915bed',
  'Author': 'WGONG <wen.g.gong@gmail.com>',
  'Date': 'Sun Jan 20 06:29:26 2019 -0500',
  'msg': 'first commit',
  'files': ['README.md']})

##  [How to list all the files in a commit?](https://stackoverflow.com/questions/424071/how-to-list-all-the-files-in-a-commit)

In [32]:
commit_id = "7a39e583c40683933767de35728f3c7d85915bed"

In [39]:
!git show 7a39e583c40683933767de35728f3c7d85915bed

commit 7a39e583c40683933767de35728f3c7d85915bed
Author: WGONG <wen.g.gong@gmail.com>
Date:   Sun Jan 20 06:29:26 2019 -0500

    first commit

diff --git a/README.md b/README.md
new file mode 100644
index 0000000..bc35f39
--- /dev/null
+++ b/README.md
@@ -0,0 +1 @@
+# hello-jenkins


In [36]:
cmd = f"git show {commit_id}"
cmd_out=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

print(cmd_out)

commit c1260ab315b782f2bc056fa40f631ef23983a987
Author: wgong <wen.g.gong@gmail.com>
Date:   Wed Jan 30 12:43:55 2019 -0500

    add date timezone

diff --git a/Jenkinsfile b/Jenkinsfile
index 2df7003..310a219 100644
--- a/Jenkinsfile
+++ b/Jenkinsfile
@@ -6,6 +6,8 @@ pipeline {
                 sh 'echo "Hello Jenkins Pipeline"'
                 sh '''
                     echo "Multiline shell steps works"
+                    date
+                    TZ=Europe/Amsterdam date +%Y-%m-%d:%H%M
                     whoami
                     pwd
                     hostname



In [40]:
# list filename only
cmd=f"git diff-tree --no-commit-id --name-only -r {commit_id}"
cmd_out=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
print(cmd_out)

Jenkinsfile



In [63]:
for log in git_log:
    commit_id = log['commit']
    files = log['files']
    print(commit_id,"\n",files)

c1260ab315b782f2bc056fa40f631ef23983a987 
 ['Jenkinsfile']
8bd0d3da888634fea4d6619ba7cbf1a19edd1df0 
 ['Jenkinsfile']
8f6e1a5faf163886270f8913478920d4c1d8149d 
 ['Jenkinsfile', 'jenkins/Jenkinsfile.7.multi-steps-docker']
69db2dddb8857eedd65171b11bc6d43469050bbf 
 ['Jenkinsfile', 'jenkins/Jenkinsfile.5.python']
7c019b636bb2242552fdcf1e4c40e8c06bf652e2 
 ['README.md']
39e4cbb67c59d0dff6d76b7c52e491d3dcfc7333 
 ['Jenkinsfile', 'jenkins/Jenkinsfile.100.CICD']
ee1017a8ea520960c6ff1640fd6e8d01e4c2c250 
 ['README.md']
f1c9e0aabe1526fda399d55ec6cfb6b5f09d6e88 
 ['README.md']
3f07508dd12515e917c41f851ae3e269ba523d1e 
 ['README.md']
473a959edc56393d5a4dfa1c82d2b113b92c3c38 
 ['Jenkinsfile', 'jenkins/Jenkinsfile.7.multi-steps']
33cf96e110e9b82ac030d4896fb74c7a2852a436 
 ['Jenkinsfile']
407ba5f5500fd571c74c2ba5ce6ba1a4938584ce 
 ['Jenkinsfile']
1fce68170453e0e544667e8e461172be3939a831 
 ['jenkins/Jenkinsfile.7.multi-steps', 'jenkins/Jenkinsfile.8.post']
e37441e00aa611d097ca43b7154edffbc91cfcfe 
 [

## [How to revert a Git repository to a previous commit](https://stackoverflow.com/questions/4114095/how-to-revert-a-git-repository-to-a-previous-commit)

In [69]:
cmd = f"git log --name-only"
cmd_out=subprocess.run(cmd.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')

git_log = parse_git_log(cmd_out)

In [72]:
git_log[0:3]

[{'commit': '0cd6ba8ae71e7ce21a05624c544036a96ce73432',
  'Author': 'wgong <wen.g.gong@gmail.com>',
  'Date': 'Fri Feb 1 00:29:24 2019 -0500',
  'msg': 'Revert "add failed testcase"',
  'files': ['    ', '    This reverts ']},
 {'commit': '6b269a67c69ae80b28c63dce25a57a5ae7949bf9.',
  'Author': '',
  'Date': 'Jenkinsfile',
  'msg': '',
  'files': []},
 {'commit': '6b269a67c69ae80b28c63dce25a57a5ae7949bf9',
  'Author': 'wgong <wen.g.gong@gmail.com>',
  'Date': 'Fri Feb 1 00:21:59 2019 -0500',
  'msg': 'add failed testcase',
  'files': ['Jenkinsfile']}]

above testcase failed, let us revert it

```
git revert HEAD
git commit -m "revert to previous commit"
git push
```